# Desafio Obrigatório

Aula 14 - Tratamento das Bases - Realizar o tratamento das bases coletadas via API do projeto final na aula 12.

#Observação: Fiz alterações nas APIs selecionadas para entrega da aula 12
e consequentemente serão as mesmas utilizadas nestre Desafio

In [1]:
#importando bibliotecas
import pandas as pd
import requests

In [2]:
#importando biblioteca de notificação e suas definições definindo
from plyer import notification 
notification.notify(
                    title='Informar o título da notificação',
                    message='Descrição da mensagem de alerta',
                    app_name='Nome do Alerta',
                    timeout=10
                    ) #Utilizado valor 10 como sugerido na atividade slide 36, aula 4 - Fundamentos de programação III

In [3]:
#Criando Alerta com notification.notify definido anteriormente
#importar datetime
from datetime import datetime

#definição do Alerta
def Alerta_default(text1):
    vNow = str(datetime.now())
    text1 = "Falha ao acessar API"
    msg_texto = f"Erro! {text1} \nOcorrência: {vNow}"

    if text1 == 1:
        text1 = "Falha ao ler API"
    else:
        print("Erro desconhecido!",text1)

    notification.notify(
            title=text1,
            message=msg_texto,
            app_name='Alerta!',
            timeout=10
                        )

In [ ]:
# Utilizando consumo da API para leitura dos dados
api_uf = "https://servicodados.ibge.gov.br/api/v1/localidades/estados"
response = requests.get(api_uf)
if response.status_code == 200:
    uf_json = response.json()
else:
    #print("Erro ao acessar a API\nErro: {}".format(response.status_code))
    Alerta_default(1)

# Converter dados em Data Frame
base_uf = pd.DataFrame(uf_json)

# Expandindo a coluna 'regiao' para detalhar o seu conteúdo em colunas separadas
base_regiao_detalhada = base_uf['regiao'].apply(pd.Series)
base_regiao_detalhada.columns = ['regiao_id', 'regiao_sigla', 'regiao_nome']

# Concatenando os DataFrames
bd_uf = pd.concat([base_uf.drop(columns=['regiao']), base_regiao_detalhada], axis=1)

# Exibir o DataFrame 
bd_uf


,id,sigla,nome,regiao_id,regiao_sigla,regiao_nome
0,11,RO,Rondônia,1,N,Norte
1,12,AC,Acre,1,N,Norte
2,13,AM,Amazonas,1,N,Norte
3,14,RR,Roraima,1,N,Norte
4,15,PA,Pará,1,N,Norte
5,16,AP,Amapá,1,N,Norte
6,17,TO,Tocantins,1,N,Norte
7,21,MA,Maranhão,2,NE,Nordeste
8,22,PI,Piauí,2,NE,Nordeste
9,23,CE,Ceará,2,NE,Nordeste


In [ ]:
# Utilizando consumo da API para leitura dos dados 
api_ranking_nomes = "https://servicodados.ibge.gov.br/api/v2/censos/nomes/ranking?localidade="
dados_api_ranking = []

# Fazendo chamadas de API para cada 'id' na coluna do DataFrame e armazenando os resultados
# Iterar sobre a coluna 'id' do DataFrame
for id in bd_uf['id']:
    api_ranking_id = api_ranking_nomes + str(id)
    response = requests.get(api_ranking_id)
    if response.status_code == 200:
        ranking_id = response.json()
        # laço criando utilizando a estrutura dos dados estar em uma lista com as chaves 'nome', 'frequencia' e 'ranking'
        for item in ranking_id:
            for record in item['res']:
                dados_api_ranking.append({
                    'localidade_id': id,
                    'nome': record['nome'],
                    'frequencia': record['frequencia'],
                    'ranking': record['ranking']
                })
    else:
        #print(f"Erro ao acessar a API para localidade {id}\nErro: {response.status_code}")
        Alerta_default(1)

# Converter a lista de dados em um DataFrame
bd_ranking = pd.DataFrame(dados_api_ranking)

# Exibir o DataFrame
bd_ranking


,localidade_id,nome,frequencia,ranking
0,11,MARIA,72579,1
1,11,JOSE,37986,2
2,11,JOAO,23366,3
3,11,ANA,20134,4
4,11,ANTONIO,16252,5
...,...,...,...,...
535,53,FRANCISCA,11858,16
536,53,BRUNO,10753,17
537,53,EDUARDO,10332,18
538,53,LUIS,10291,19
